In [43]:
# Load chunked text file into a list of chunks
with open("all_chunks.txt", "r", encoding="utf-8") as f:
    text_data = f.read()

# Split based on double newlines (assuming chunks were separated like that)
texts = text_data.strip().split("\n\n")



In [49]:
from nomic import embed
import pickle

# Step 1: Load the context-based chunks
with open("context_chunks.txt", "r", encoding="utf-8") as f:
    text_data = f.read()

# Step 2: Split back into list of chunks
texts = text_data.strip().split("\n\n")

print(f"📄 Loaded {len(texts)} chunks.")

# Step 3: Generate embeddings using Nomic
result = embed.text(texts, model='nomic-embed-text-v1')
embeddings = result['embeddings']

print(f"✅ Generated {len(embeddings)} embeddings.")

# Step 4: Save to .pkl file
with open("pu_embeddings.pkl", "wb") as f:
    pickle.dump({"texts": texts, "embeddings": embeddings}, f)

print("💾 Saved embeddings to pu_embeddings.pkl")


📄 Loaded 11826 chunks.
✅ Generated 11826 embeddings.
💾 Saved embeddings to pu_embeddings.pkl


In [50]:
def semantic_search(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)[0]
    scores = np.dot(chunk_embeddings, query_embedding)
    top_indices = scores.argsort()[::-1][:top_k]
    return [chunks[i] for i in top_indices]


In [51]:
import requests
def ask_ollama(query, model_name="llama3.2:3b", top_k=5):
    top_chunks = semantic_search(query, top_k)
    context = "\n\n".join(top_chunks)

    prompt = f"""Use the following context to answer the question:

{context}

Question: {query}

Answer:"""

    print("Final Prompt Sent to Ollama:\n")
    print(prompt)

    try:
        response = requests.post("http://localhost:11434/api/generate", json={
            "model": model_name,
            "prompt": prompt,
            "stream": False
        })

        print("Raw Response:\n")
        print(response.json())

        print("\nFinal Answer:\n")
        print(response.json().get("response", "[No response]"))

    except Exception as e:
        print(f"Error: {e}")




In [52]:
ask_ollama("What is the eligibility for MSc Chemistry?")

Final Prompt Sent to Ollama:

Use the following context to answer the question:

Chunk 51
M.Tech. (Polymer)#
15+5 Part-
time+2 NRI+ 1
Foreign
National
2 years
B.E./B.Tech.(Chemical) 04 years or
Five Year Integrated B.E.(Chem.)-
MBA at least 60% marks in the
aggregate from Panjab University
or any other University recognized
by Panjab University as equivalent
thereto. Mode of Admission: Preferences will
be
given
to
GATE
qualified
candidates. Candidate appearing for
PU-CET (PG) will be given admission
if some seats remain vacant after
exhausting the list of GATE qualified
candidates. FACULTY OF ENGINEERING AND TECHNOLOGY
71
HANDBOOK OF INFORMATION 2022
OR
Master’s
degree
in
Technical
Chemistry/Applied
Chemistry/
Industrial
Chemistry/Chemistry
(with
Mathematics
upto
graduation)
or
an
equivalent
examination) with at least 55%
marks in aggregate from Panjab
University (in case students are
must have the conversion formula
issued by the concerned University
or head of the department) or any


In [55]:
# app.py
import streamlit as st
import requests
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load embeddings
with open("pu_embeddings.pkl", "rb") as f:
    data = pickle.load(f)

texts = data["texts"]
embeddings = np.array(data["embeddings"])

# Semantic search
def retrieve_context(query, top_k=3):
    model = SentenceTransformer("all-MiniLM-L6-v2")
    query_embedding = model.encode([query])
    scores = cosine_similarity(query_embedding, embeddings)[0]
    top_indices = np.argsort(scores)[-top_k:][::-1]
    return [texts[i] for i in top_indices]

# LLM call to Ollama
def ask_llama(query, history):
    context_chunks = retrieve_context(query)
    context = "\n\n".join(context_chunks)
    history_text = "\n".join([f"User: {q}\nAssistant: {a}" for q, a in history])

    prompt = f"""You are a helpful assistant for Panjab University admissions.
Use the context and past conversation to answer questions.

Context:
{context}

Conversation so far:
{history_text}

Current question:
User: {query}
Assistant:"""

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "llama3", "prompt": prompt, "stream": False}
    )

    return response.json()["response"].strip()

# Streamlit UI
st.title("🎓 PU Admissions RAG Chatbot")

if "history" not in st.session_state:
    st.session_state.history = []

user_input = st.chat_input("Ask me about admissions...")

if user_input:
    with st.spinner("Thinking..."):
        answer = ask_llama(user_input, st.session_state.history)
        st.session_state.history.append((user_input, answer))

# Display chat history
for q, a in st.session_state.history:
    st.chat_message("user").write(q)
    st.chat_message("assistant").write(a)


2025-07-02 15:23:09.396 
  command:

    streamlit run C:\Users\ankus\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-07-02 15:23:09.396 Session state does not function when running a script without `streamlit run`
